#### Downloading & Inspecting EUMETSAT/MSG Data

Code adapted from https://eumetsatspace.atlassian.net/wiki/spaces/EUMDAC/pages/1760198661/Python+Library

Requires signing up for an account with EUMETSAT EO Portal: https://eoportal.eumetsat.int/userMgmt/
Once done, the consumer key and secret can be found under the user information: https://eumetsatspace.atlassian.net/wiki/spaces/DSEV/pages/375652353/API+Authentication

In [2]:
import os
import xarray as xr

import eumdac
import datetime
import shutil

In [4]:
# Insert your personal key and secret into the single quotes
consumer_key = ""
consumer_secret = ""

credentials = (consumer_key, consumer_secret)

token = eumdac.AccessToken(credentials)

print(f"This token '{token}' expires {token.expiration}")

This token '336ec30b-c70c-3bfe-94f1-7551e89726b9' expires 2024-02-22 13:37:16.972039


In [6]:
datastore = eumdac.DataStore(token)
datastore.collections

[<class 'eumdac.collection.Collection'>(EO:EUM:DAT:MSG:CLM-IODC),
 <class 'eumdac.collection.Collection'>(EO:EUM:DAT:0082),
 <class 'eumdac.collection.Collection'>(EO:EUM:DAT:0237),
 <class 'eumdac.collection.Collection'>(EO:EUM:DAT:0241),
 <class 'eumdac.collection.Collection'>(EO:EUM:DAT:0274),
 <class 'eumdac.collection.Collection'>(EO:EUM:DAT:0855),
 <class 'eumdac.collection.Collection'>(EO:EUM:DAT:0857),
 <class 'eumdac.collection.Collection'>(EO:EUM:DAT:0584),
 <class 'eumdac.collection.Collection'>(EO:EUM:DAT:0412),
 <class 'eumdac.collection.Collection'>(EO:EUM:DAT:MSG:MSG15-RSS),
 <class 'eumdac.collection.Collection'>(EO:EUM:DAT:MULT:HIRSL1),
 <class 'eumdac.collection.Collection'>(EO:EUM:DAT:0836),
 <class 'eumdac.collection.Collection'>(EO:EUM:DAT:METOP:ASCSZF1B),
 <class 'eumdac.collection.Collection'>(EO:EUM:DAT:METOP:ASCSZR1B),
 <class 'eumdac.collection.Collection'>(EO:EUM:DAT:0882),
 <class 'eumdac.collection.Collection'>(EO:EUM:DAT:0394),
 <class 'eumdac.collection.C

In [9]:
selected_collection = datastore.get_collection('EO:EUM:DAT:MSG:HRSEVIRI')
selected_collection?

Type:        Collection
String form: EO:EUM:DAT:MSG:HRSEVIRI
File:        /opt/anaconda3/lib/python3.9/site-packages/eumdac/collection.py
Docstring:  
Collection in the Data Store

Attributes:
    datastore: Reference to the Data Store

Arguments:
    collection_id: Data Store ID of the collection
    datastore: Reference to the Data Store


##### Define download parameters & compile list of products

In [11]:
# Add vertices for polygon, wrapping back to the start point.
geometry = [[-1.0, -1.0],[4.0, -4.0],[8.0, -2.0],[9.0, 2.0],[6.0, 4.0],[1.0, 5.0],[-1.0, -1.0]]

# Set sensing start and end time
start = datetime.datetime(2021, 10, 10, 9, 0)
end = datetime.datetime(2021, 10, 10, 10, 0)

# Retrieve datasets that match our filter
products = selected_collection.search(
    geo='POLYGON(({}))'.format(','.join(["{} {}".format(*coord) for coord in geometry])),
    dtstart=start, 
    dtend=end)
    
print(f'Found Datasets: {len(products)} datasets for the given time range')

for product in products:
    print(str(product))

Found Datasets: 4 datasets for the given time range
MSG4-SEVI-MSG15-0100-NA-20211010095743.941000000Z-NA
MSG4-SEVI-MSG15-0100-NA-20211010094243.725000000Z-NA
MSG4-SEVI-MSG15-0100-NA-20211010092743.511000000Z-NA
MSG4-SEVI-MSG15-0100-NA-20211010091243.299000000Z-NA


##### Download products

Each product contains three files (EOPMetadata.xml, manifest.xml, and data.nat files). When downloading the whole product, a .zip folder is created.

In [12]:
for product in products:
    with product.open() as fsrc, \
            open(fsrc.name, mode='wb') as fdst:
        shutil.copyfileobj(fsrc, fdst)
        print(f'Download of product {product} finished.')
print('All downloads are finished.')

Download of product MSG4-SEVI-MSG15-0100-NA-20211010095743.941000000Z-NA finished.
Download of product MSG4-SEVI-MSG15-0100-NA-20211010094243.725000000Z-NA finished.
Download of product MSG4-SEVI-MSG15-0100-NA-20211010092743.511000000Z-NA finished.
Download of product MSG4-SEVI-MSG15-0100-NA-20211010091243.299000000Z-NA finished.
All downloads are finished.


In [24]:
for product in products:    
    print(product.url)

https://api.eumetsat.int/data/download/1.0.0/collections/EO%3AEUM%3ADAT%3AMSG%3AHRSEVIRI/products/MSG4-SEVI-MSG15-0100-NA-20211010095743.941000000Z-NA?access_token=2d8ce283-d340-3e02-8905-641e42ca1dfb
https://api.eumetsat.int/data/download/1.0.0/collections/EO%3AEUM%3ADAT%3AMSG%3AHRSEVIRI/products/MSG4-SEVI-MSG15-0100-NA-20211010094243.725000000Z-NA?access_token=2d8ce283-d340-3e02-8905-641e42ca1dfb
https://api.eumetsat.int/data/download/1.0.0/collections/EO%3AEUM%3ADAT%3AMSG%3AHRSEVIRI/products/MSG4-SEVI-MSG15-0100-NA-20211010092743.511000000Z-NA?access_token=2d8ce283-d340-3e02-8905-641e42ca1dfb
https://api.eumetsat.int/data/download/1.0.0/collections/EO%3AEUM%3ADAT%3AMSG%3AHRSEVIRI/products/MSG4-SEVI-MSG15-0100-NA-20211010091243.299000000Z-NA?access_token=2d8ce283-d340-3e02-8905-641e42ca1dfb


##### Download only selected products & files

In [25]:
selected_product = datastore.get_product(
    product_id='MSG4-SEVI-MSG15-0100-NA-20211110081242.766000000Z-NA',
    collection_id='EO:EUM:DAT:MSG:HRSEVIRI')

The data is stored in a .nat file

In [42]:
for entry in selected_product.entries:
    if entry.endswith('nat'):
        print(entry)

MSG4-SEVI-MSG15-0100-NA-20211110081242.766000000Z-NA.nat


In [43]:
try:
    with selected_product.open(entry='MSG4-SEVI-MSG15-0100-NA-20211110081242.766000000Z-NA.nat') as fsrc, \
            open(fsrc.name, mode='wb') as fdst:
        shutil.copyfileobj(fsrc, fdst)
        print(f'Download of file {fsrc.name} finished.')
except eumdac.product.ProductError as error:
    print(f"Error related to the product '{selected_product}' while trying to download it: '{error.msg}'")

Download of file MSG4-SEVI-MSG15-0100-NA-20211110081242.766000000Z-NA.nat finished.


Unfortunately, .nat files are a bit complicated to read/load. One of the best packages is satpy, which then allows loading the files using xarray.

In [85]:
from satpy import Scene
scn = Scene(reader="seviri_l1b_native", filenames=['MSG4-SEVI-MSG15-0100-NA-20211110081242.766000000Z-NA.nat'])

In [86]:
# MSG data contains 12 channels, including the high-resolution visible (HRV) channel.
datasets = scn.available_dataset_names()
datasets

['HRV',
 'IR_016',
 'IR_039',
 'IR_087',
 'IR_097',
 'IR_108',
 'IR_120',
 'IR_134',
 'VIS006',
 'VIS008',
 'WV_062',
 'WV_073']

In order to convert the scn object to xarray, each dataset needs to be loaded.

In [89]:
scn.load(datasets[1:], generate=False)
# Note: The HRV channel comes in 1 km resolution, while the other channels are 3 km resolution.
# Because of the size mismatch, the HRV channel would either need to be downscaled,
# or it needs to the loaded separetely to the other channels


In [90]:
ds = scn.to_xarray()

In [96]:
ds

<xarray.Dataset>
Dimensions:             (y: 3712, x: 3712)
Coordinates: (12/15)
    IR_016_acq_time     (y) datetime64[ns] NaT NaT NaT NaT ... NaT NaT NaT NaT
  * y                   (y) float64 -5.566e+06 -5.563e+06 ... 5.569e+06
  * x                   (x) float64 5.566e+06 5.563e+06 ... -5.569e+06
    longitude           (y, x) float64 dask.array<chunksize=(928, 3712), meta=np.ndarray>
    latitude            (y, x) float64 dask.array<chunksize=(928, 3712), meta=np.ndarray>
    IR_039_acq_time     (y) datetime64[ns] NaT NaT NaT NaT ... NaT NaT NaT NaT
    ...                  ...
    IR_120_acq_time     (y) datetime64[ns] NaT NaT NaT NaT ... NaT NaT NaT NaT
    IR_134_acq_time     (y) datetime64[ns] NaT NaT NaT NaT ... NaT NaT NaT NaT
    VIS006_acq_time     (y) datetime64[ns] NaT NaT NaT NaT ... NaT NaT NaT NaT
    VIS008_acq_time     (y) datetime64[ns] NaT NaT NaT NaT ... NaT NaT NaT NaT
    WV_062_acq_time     (y) datetime64[ns] NaT NaT NaT NaT ... NaT NaT NaT NaT
    WV_073_acq_time     (y) datetime64[ns] NaT NaT NaT NaT ... NaT NaT NaT NaT
Data variables:
    msg_seviri_fes_3km  int64 0
    IR_016              (y, x) float32 dask.array<chunksize=(928, 3712), meta=np.ndarray>
    IR_039              (y, x) float32 dask.array<chunksize=(928, 3712), meta=np.ndarray>
    IR_087              (y, x) float32 dask.array<chunksize=(928, 3712), meta=np.ndarray>
    IR_097              (y, x) float32 dask.array<chunksize=(928, 3712), meta=np.ndarray>
    IR_108              (y, x) float32 dask.array<chunksize=(928, 3712), meta=np.ndarray>
    IR_120              (y, x) float32 dask.array<chunksize=(928, 3712), meta=np.ndarray>
    IR_134              (y, x) float32 dask.array<chunksize=(928, 3712), meta=np.ndarray>
    VIS006              (y, x) float32 dask.array<chunksize=(928, 3712), meta=np.ndarray>
    VIS008              (y, x) float32 dask.array<chunksize=(928, 3712), meta=np.ndarray>
    WV_062              (y, x) float32 dask.array<chunksize=(928, 3712), meta=np.ndarray>
    WV_073              (y, x) float32 dask.array<chunksize=(928, 3712), meta=np.ndarray>
Attributes:
    history:      Created by pytroll/satpy on 2024-02-22 16:02:51.041901
    Conventions:  CF-1.7